# 1. linux 内核模块
linux 内核模块三要素：入口、出口、许可证

## 1.1 模块编写基本

In [ ]:
#include <linux/init.h>
#include <linux/module.h>
// 入口
// static :限定作用域
// int    :函数的返回值类型
// __init :内核中定义的宏，#define __init __section(".init.text") 
// .init.text是一个段，在内核链接脚本（vmlinux.lds）中可以找到它
// __init告诉编译器将demo_init函数放在.init.text段中
// demo_init:入口函数的名字，习惯上入口命名格式：led_init adc_init uart_init...
// (void) :没有参数
static int __init demo_init(void)
{
    return 0;
}
static void __exit demo_exit(void)
{
}
module_init(demo_init); //告诉内核入口地址
module_exit(demo_exit); //告诉内核出口地址
MODULE_LICENSE("GPL");  // 许可证

## 1.2 接口

In [ ]:
/**
 * @功能：打印函数
 * @参数：fmt 格式化字符串
 * @返回值：打印字符的个数
 * @备注：可变参数
*/
int printk(const char *fmt, ...);

In [ ]:
/**
 * @功能：接受命令的参数
 * @参数：name  参数名， type 参数类型， perm 参数权限
 *      @type：Standard types are:
 *	            byte, hexint, short, ushort, int, uint, long, ulong
 *	            charp: a character pointer
 *	            bool: a bool, values 0/1, y/n, Y/N.
 *	            invbool: the above, only sense-reversed (N = true).
 * @返回值：无
 * @备注：无
*/
moudle_param(name, type, perm)

/**
 * @功能：对变量进行描述
 * @参数：_parm 变量名， desc：描述
 * @返回值：无
 * @备注：无
*/
MODULE_PARM_DESC(_parm, desc)

/**
 * @功能：导出符号表
 * @参数：add 函数名
 * @返回值：无
 * @备注：无
*/
EXPORT_SYMBOL_GPL(add)

# 2. 字符设备驱动
按照字节流访问，只能顺序访问，不能无序访问的设备。

## 2.1 接口

In [ ]:
#include <linux/fs.h>
/**
 * @功能：注册字符设备驱动
 * @参数：major：主设备号，name：设备名，fops：文件操作集
 * @返回值：成功返回0，失败返回负值
 * @备注：如果major为0，则由系统自动分配主设备号
*/
int register_chrdev(unsigned int major, const char *name, const struct file_operations *fops)

/**
 * @功能：注销字符设备驱动
 * @参数：major：主设备号，name：设备名
 * @返回值：无
 * @备注：无
*/
void unregister_chrdev(unsigned int major, const char *name)

In [ ]:
#include <linux/uaccess.h>
/**
 * @功能：从用户空间拷贝数据到内核空间
 * @参数：to：目标地址，from：源地址，n：拷贝字节数
 * @返回值：成功返回0，失败返回拷贝失败的字节数
 * @备注：无
*/
unsigned long copy_from_user(void *to, const void __user *from, unsigned long n)

/**
 * @功能：从内核空间拷贝数据到用户空间
 * @参数：to：目标地址，from：源地址，n：拷贝字节数
 * @返回值：成功返回0，失败返回拷贝失败的字节数
 * @备注：无
*/
unsigned long copy_from_user(void *to, const void __user *from, unsigned long n)

In [ ]:
#include <linux/io.h>
/**
 * @功能：将物理地址映射到内核虚拟地址空间
 * @参数：phy_addr：物理地址，size：映射字节数
 * @返回值：映射成功返回内核虚拟地址，失败返回NULL
 * @备注：无
*/
void  *ioremap(resource_size_t phy_addr, size_t size)

/**
 * @功能：解除内核虚拟地址空间的映射
 * @参数：virt_addr：内核虚拟地址
 * @返回值：无
 * @备注：无
*/
void iounmap(volatile void  *virt_addr)

In [ ]:
#include <linux/device.h>
/**
 * @功能：创建一个class
 * @参数：owner：模块的拥有者，name：class的名字
 * @返回值：成功返回class指针，失败返回NULL
 * @备注：无
*/
struct class * class_create(owner, name)
/**
 * @功能：注销一个class
 * @参数：cls：class指针
 * @返回值：无
 * @备注：无
*/
void class_destroy(struct class *cls)

/**
 * @功能：创建一个device
 * @参数：class：class指针，parent：父设备指针，devt：设备号，drvdata：驱动数据，fmt：设备名，...：变参
 * @返回值：成功返回device指针，失败返回NULL
 * @备注：fmt,... - 设备名，如 "myled%d"，%d - myled0, myled1, ...
*/
struct device *device_create(struct class *class, struct device *parent, dev_t devt, void *drvdata, const char *fmt, ...)
/**
 * @功能：注销一个device
 * @参数：class：class指针，devt：设备号
 * @返回值：无
 * @备注：无
*/
void device_destroy(struct class *class, dev_t devt)

In [ ]:
#include <linux/cdev.h>
/**
 * @功能：分配一个cdev对象
 * @参数：无
 * @返回值：成功返回cdev指针，失败返回NULL
 * @备注：无
*/
struct cdev *cdev_alloc(void)

/**
 * @功能：初始化一个cdev对象
 * @参数：cdev：cdev指针，fops：文件操作集
 * @返回值：无
 * @备注：无
*/
void cdev_init(struct cdev *cdev, const struct file_operations *fops)

/**
 * @功能：添加一个cdev对象到系统
 * @参数：p：cdev指针，dev：设备号，count：设备号个数
 * @返回值：成功返回0，失败返回负值
 * @备注：无
*/
int cdev_add(struct cdev *p, dev_t dev, unsigned count)

/**
 * @功能：注册指定的设备号
 * @参数：from: 起始设备号，count：设备号个数，name：设备名
 * @返回值：成功返回0，失败返回负值
 * @备注：主设备号不能超过511，
*/
int register_chrdev_region(dev_t from, unsigned count, const char *name)
/**
 * @功能：注销指定的设备号
 * @参数：from: 起始设备号，count：设备号个数
 * @返回值：无
 * @备注：无
*/
void unregister_chrdev_region(dev_t from, unsigned count)

/**
 * @功能: 动态注册设备号
 * @参数：dev：申请到的设备号指针，firstminor：起始次设备号，count：设备号个数，name：设备名
 * @返回值：成功返回0，失败返回负值
 * @备注：无
*/
int alloc_chrdev_region(dev_t *dev, unsigned int firstminor, unsigned int count, const char *name)


## 2.2 结构体

In [ ]:
// 文件操作结构体
struct file_operations {
    struct module *owner;
    loff_t (*llseek) (struct file *, loff_t, int);
    ssize_t (*read) (struct file *, char __user *, size_t, loff_t *);
    ssize_t (*write) (struct file *, const char __user *, size_t, loff_t *);
    ssize_t (*aio_read) (struct kiocb *, const struct iovec *, unsigned long, loff_t);
    ssize_t (*aio_write) (struct kiocb *, const struct iovec *, unsigned long, loff_t);
    int (*readdir) (struct file *, void *, filldir_t);
    unsigned int (*poll) (struct file *, struct poll_table_struct *);
    long (*unlocked_ioctl) (struct file *, unsigned int, unsigned long);
    long (*compat_ioctl) (struct file *, unsigned int, unsigned long);
    int (*mmap) (struct file *, struct vm_area_struct *);
    int (*open) (struct inode *, struct file *);
    int (*flush) (struct file *, fl_owner_t id);
    int (*release) (struct inode *, struct file *);
    int (*fsync) (struct file *, loff_t, loff_t, int datasync);
    int (*aio_fsync) (struct kiocb *, int datasync);
    int (*fasync) (int, struct file *, int);
    int (*lock) (struct file *, int, struct file_lock *);
    ssize_t (*sendpage) (struct file *, struct page *, int, size_t, loff_t *, int);
    unsigned long (*get_unmapped_area)(struct file *, unsigned long, unsigned long, unsigned long, unsigned long);
    int (*check_flags)(int);
    int (*flock) (struct file *, int, struct file_lock *);
    ssize_t (*splice_write)(struct pipe_inode_info *, struct file *, loff_t *, size_t, unsigned int);
    ssize_t (*splice_read)(struct file *, loff_t *, struct pipe_inode_info *, size_t, unsigned int);
    int (*setlease)(struct file *, long, struct file_lock **, void **);
    long (*fallocate)(struct file *file, int mode, loff_t offset, loff_t len);
};

In [ ]:
// 文件结构体
struct file {
    struct path     f_path;
    struct inode    *f_inode;       /* cached value */
    const struct file_operations    *f_op;
    spinlock_t      f_lock;         /* f_ep_links, f_flags, no IRQ */
    atomic_long_t   f_count;
    unsigned int    f_flags;
    fmode_t         f_mode;
    loff_t          f_pos;
    struct fown_struct f_owner;
    const struct cred *f_cred;
    struct file_ra_state    f_ra;
    u64             f_version;
#ifdef CONFIG_SECURITY
    void            *f_security;
#endif
    /* needed for tty driver, and maybe others */
    void            *private_data;
#ifdef CONFIG_EPOLL
    /* Used by fs/eventpoll.c to link all the hooks to this file */
    struct list_head    f_ep_links;
    struct list_head    f_tfile_llink;
#endif /* #ifdef CONFIG_EPOLL */
    struct address_space    *f_mapping;
#ifdef CONFIG_DEBUG_WRITECOUNT
    unsigned long f_mnt_write_state;
#endif
};

# 3. 内核中的并发和竞态
当多个进程通过同一个驱动（临界资源）访问硬件的时候，竞态就会产生。有 5 种解决方案来优化程序。

## 3.1 中断屏蔽
暂时的关闭中断，只针对单核处理器有效。

In [ ]:
local_irq_disable();
// 临界区
local_irq_enable();

## 3.2 自旋锁（忙等锁）
除首次获得锁之外的进程尝试获取锁会进入自旋状态直到获得锁的进程释放锁。

In [ ]:
// 由于自旋锁内的代码不能太长，所以常用来加锁一个信号量
int flag;
// 1. 定义一个自旋锁
spinlock_t my_lock;
// 2. 初始化自旋锁
/**
 * @功能：初始化自旋锁
 * @参数：lock：自旋锁指针
 * @返回值：无
 * @备注：无
*/
void spin_lock_init(spinlock_t *lock);
// 3. 加锁
/**
 * @功能：加锁
 * @参数：lock：自旋锁指针
 * @返回值：无
 * @备注：无
*/
void spin_lock(spinlock_t *lock);
// 4. 解锁
/**
 * @功能：解锁
 * @参数：lock：自旋锁指针
 * @返回值：无
 * @备注：无
*/
void spin_unlock(spinlock_t *lock);

## 3.3 信号量
除首次获得信号量之外的进程尝试获取信号量会进入休眠状态直到获得获得信号量的进程释放信号量，休眠不消耗消耗 CPU 资源。

In [ ]:
// 1. 定义一个信号量
struct semaphore my_sem;
// 2. 初始化信号量
/**
 * @功能：初始化信号量
 * @参数：sem：信号量指针，val：信号量初始值
 * @返回值：无
 * @备注：val一般为1，表示只有一个进程可以访问临界区
*/
void sema_init(struct semaphore *sem, int val);
// 3. 加锁
/**
 * @功能：加锁
 * @参数：sem：信号量指针
 * @返回值：无
 * @备注：让信号量的值减1，如果值为0，则休眠等待
*/
void down(struct semaphore *sem);
// 4. 解锁
/**
 * @功能：解锁
 * @参数：sem：信号量指针
 * @返回值：无
 * @备注：让信号量的值加1，如果值为0，则唤醒等待的进程
*/
void up(struct semaphore *sem);
// 5. 尝试加锁
/**
 * @功能：尝试加锁
 * @参数：sem：信号量指针
 * @返回值：成功返回0，失败返回1
 * @备注：不会休眠，相当于非阻塞
*/
int down_trylock(struct semaphore *sem);

## 3.4 互斥体
互斥体与信号量的区别是在获取不到资源的情况下，会稍微等一会再进入休眠。  
通常在不确定临界区大小的时候，优先使用互斥体。

In [ ]:
// 1. 定义一个互斥锁
struct mutex my_mutex;
// 2. 初始化互斥锁
/**
 * @功能：初始化互斥锁
 * @参数：lock：互斥锁指针
 * @返回值：无
 * @备注：无
*/
void mutex_init(struct mutex *lock);
// 3. 加锁
/**
 * @功能：加锁
 * @参数：lock：互斥锁指针
 * @返回值：无
 * @备注：无
*/
void mutex_lock(struct mutex *lock);
// 4. 解锁
/**
 * @功能：解锁
 * @参数：lock：互斥锁指针
 * @返回值：无
 * @备注：无
*/
void mutex_unlock(struct mutex *lock);
// 5. 尝试加锁
/**
 * @功能：尝试加锁
 * @参数：lock：互斥锁指针
 * @返回值：成功返回1，失败返回0
 * @备注：无
*/
int mutex_trylock(struct mutex *lock);

## 3.5 原子操作
相当于将整个操作看成一个不可被分割的状态，只要开始执行就必须一次执行完。对应原子变量同时只能有一个核处理。在一个核上运行的时候修改 counter 是通过内联汇编完成的。

In [ ]:
typedef struct {
    int counter;
} atomic_t;

In [ ]:
// 1. 定义一个原子变量
atomic_t my_atomic;
// 2. 初始化原子变量
ATOMIC_INIT(my_atomic);
// 3. 读取原子变量的值
/**
 * @功能：读取原子变量的值
 * @参数：v：原子变量指针
 * @返回值：原子变量的值
 * @备注：无
*/
int atomic_read(atomic_t *v);
// 4. 设置原子变量的值
/**
 * @功能：设置原子变量的值
 * @参数：v：原子变量指针，i：要设置的值
 * @返回值：无
 * @备注：无
*/
void atomic_set(atomic_t *v, int i);
// 5. 原子变量加1
/**
 * @功能：原子变量加1
 * @参数：v：原子变量指针
 * @返回值：无
 * @备注：无
*/
void atomic_inc(atomic_t *v);
// 6. 原子变量减1
/**
 * @功能：原子变量减1
 * @参数：v：原子变量指针
 * @返回值：无
 * @备注：无
*/
void atomic_dec(atomic_t *v);
// 

# 4. IO模型

## 4.1 非阻塞IO
调用函数读取数据的时候，不管数据是否准备好函数都会立即返回。

## 4.2 阻塞IO模型
调用函数读取数据的时候，如果数据准备好了就会立即返回，如果数据没有准备好则进程休眠。

In [ ]:
// 1. 定义信号
unsigned int flag;

// 2. 定义等待队列头
wait_queue_head_t my_queue;

// 3. 初始化等待队列头
/**
 * @功能：初始化等待队列头
 * @参数：q：等待队列头指针
 * @返回值：无
 * @备注：无
*/
void init_waitqueue_head(wait_queue_head_t *q);

// 4. 进程加入等待队列
/**
 * @功能：进程加入等待队列
 * @参数：q：等待队列头指针，flag：信号
 * @返回值：无
 * @备注：无
*/
void wait_event(wait_queue_head_t *q, unsigned int flag);

// 5. 进程加入等待队列，超时
wait_event_timeout(my_queue, flag, 1000);

// 6. 进程加入等待队列，中断
/**
 * @功能：进程加入等待队列，中断
 * @参数：q：等待队列头指针，flag：信号
 * @返回值：无
 * @备注：无
*/
int wait_event_interruptible(wait_queue_head_t *q, unsigned int flag);

// 7. 进程加入等待队列，中断，超时
/**
 * @功能：进程加入等待队列，中断，超时
 * @参数：q：等待队列头指针，flag：信号，timeout：超时时间
 * @返回值：无
*/
int wait_event_interruptible_timeout(wait_queue_head_t *q, unsigned int flag, long timeout);

// 8. flag置为1
flag = 1;

// 9. 唤醒等待队列头
/**
 * @功能：唤醒等待队列头
 * @参数：q：等待队列头指针
 * @返回值：无
*/
void wake_up(wait_queue_head_t *q);

// 10. 唤醒等待队列头，中断
/**
 * @功能：唤醒等待队列头，中断
 * @参数：q：等待队列头指针
 * @返回值：无
*/
void wake_up_interruptible(wait_queue_head_t *q);


## 4.3 IO多路复用
在同一个app应用程序中如果想要同时监听多个fd对应数据。就需要使用select/poll/epoll来完成监听。如果所有的文件描述符的数据都没有准备好，此时进程休眠。如果有一个或者多个硬件的数据准备好就会，产生硬件中断，在处理函数中唤醒休眠的进程。此时select/poll/epoll就会返回，从就绪的表中找到准备好数据的文件描述符，然后调用read将数据读取到用户空间即可。

In [ ]:
// 1. 定义返回值
__poll_t mask = 0;

// 2. 阻塞等待
/**
 * @ 功能：阻塞等待
 * @ 参数：file 文件指针, my_queue 等待队列, wait 等待条件
 * @ 返回值：无
*/
poll_wait(file, &my_queue, wait);

// 3. 判断是否可读
if (flag) {
    mask |= POLLIN | POLLRDNORM; //EPOLLIN 读  EPOLLOUT写
}

// 4. 返回
return mask;

# 5. 设备树
对于相同SOC的不同板卡，只需更换设备树文件.dtb即可实现不同板卡的无差异支持

## 5.1 设备树语法
设备树是节点和属性组成的树状结构。属性是键值对，节点可以同时包含属性和子节点。

### 5.1.1 节点
`<name>[@<unit-address>]`   
`name`:节点的类型名，name不能够超过31个字符   
`unit-address`:节点内的地址，如果节点内有地址信息就可以填充，如果没有地址信息省略   
1. 节点可以取别名
    ```c
    node:mynode@0x12345678{
        string1 = "aaaa";
    };
    //node就是mynode@0x12345678的别名
    ```
2. 节点可以被引用
    ```c
    &node{
    string1 = "bbbb";
    };
    ```
3. 节点可以合并（同名节点合并）
    ```c
    mynode@0x12345678{
        string1 = "aaaa";
        string2 = "bbbb";
    };
    ```
4. 节点内属性可以删除
    ```c
    /delete-property/属性名;
    ```

### 5.1.2 属性格式
* 文本字符串(可以有空格)用双引号表示:   
`string-property = "a string";`
* Cells'是由尖括号分隔的32位无符号整数:   
`cell-property = <0xbeef 123 0xabcd1234>;`
* 十六进制表示的单字节的数用方括号分隔:   
`binary-property = [00 01 23 45 67];`
* 混合类型可以用逗号连接在一起:   
`mixed-property = "a string", [0x01 0x23 0x45 0x67], <0x12345678>;`
* 逗号也用于创建字符串列表:   
`string-list = "red fish", "blue fish";`

### 5.1.3 语法示例

In [ ]:
/dts-v1/; // 设备树版本

/ {   //   /{};  设备树的根节点，所有的节点写在根节点之内
    node1 {  // node1{};  根节点的子节点
        a-string-property = "A string";
        a-string-list-property = "first string", "second string";
        a-byte-data-property = [01 23 34 56];
        child-node1 {  //node1的子节点
            first-child-property;  //空属性，标识作用
            second-child-property = <1>;
            a-string-property = "Hello, world";
        };
        child-node2 { //node1的子节点
        };
    };
    node2 {  //根节点子节点
        an-empty-property;
        a-cell-property = <1 2 3 4>; /* each number (cell) is a uint32 */
        child-node1 {
        };
    };
};

## 5.2 设备树节点获取

In [ ]:
// 1.device_node结构体
// 设备树的每一个节点都是通过device_node结构体来描述的
struct device_node {
    const char *name;  //设备树节点的名字  "mynode"
    const char *full_name; //设备树节点的全名 "mynode@0x12345678"
    struct property *properties; //属性的结构体
    struct device_node *parent;  //指向父节点
    struct device_node *child;   //指向子节点
    struct device_node *sibling; //指向兄弟节点
};
// 2.property结构体
// 在设备树节点内，所有的属性都是通过property节点描述的，当前节点内属性构成一条property链表
struct property {
    char *name; //属性名 “键”
    int length;     //值的长度（单位字节）
    void *value; //属性值 “值”
    struct property *next; //指向下一个property结构体
}
// 3.设备树节点获取函数
/**
 * @功能：根据节点路径获取节点
 * @参数：path：节点路径 "/mynode@0x12345678"
 * @返回值：成功返回节点首地址，失败返回NULL
 * @备注：无
*/
struct device_node *of_find_node_by_path(const char *path)

/**
 * @功能：根据节点名字获取节点
 * @参数：from：NULL，表示从根节点开始解析，name：节点名
 * @返回值：成功返回节点首地址，失败返回NULL
 * @备注：无
*/
struct device_node *of_find_node_by_name(struct device_node *from,
    const char *name)

/**
 * @功能：根据compatible获取节点
 * @参数：from：NULL，表示从根节点开始解析，type：NULL，compatible：厂商名，设备名
 * @返回值：成功返回节点首地址，失败返回NULL
 * @备注：无
*/
struct device_node *of_find_compatible_node(struct device_node *from,
    const char *type, const char *compatible)
// 4.设备树属性解析函数
/**
 * @功能：获取属性
 * @参数：np：节点指针，name：键，lenp：获取到的值的长度
 * @返回值：成功返回属性结构体地址，失败返回NULL
 * @备注：无
*/
struct property *of_find_property(const struct device_node *np,
    const char *name,int *lenp)

__b32_to_cpup() //将大端的数据转换为小端数据

# 6. GPIO 子系统

## 6.1 接口

In [ ]:
/**
 * @功能：解析得到gpio号
 * @参数：np：节点指针，propname：键名，index：键对应值的下标
 * @返回值：成功返回gpio号，失败返回错误码
 * @备注：无
*/
int of_get_named_gpio(struct device_node *np,
 const char *propname, int index)

/**
 * @功能：申请要使用的gpio
 * @参数：gpio：gpio编号（设备树），label:名字（NULL）
 * @返回值：成功返回0，失败返回错误码
 * @备注：无
*/
int gpio_request(unsigned gpio, const char *label)

/**
 * @功能：设置管脚的方向为输入
 * @参数：gpio：gpio编号
 * @返回值：成功返回0，失败返回错误码
 * @备注：无
*/
int gpio_direction_input(unsigned gpio)

/**
 * @功能：设置管脚的方向为输出
 * @参数：gpio：gpio编号，value：1高  0低电平
 * @返回值：成功返回0，失败返回错误码
 * @备注：无
*/
int gpio_direction_output(unsigned gpio, int value) 

/**
 * @功能：设置电平的状态
 * @参数：gpio：gpio编号，value：1高  0低电平
 * @返回值：无
 * @备注：无
*/
void gpio_set_value(unsigned gpio, int value)

/**
 * @功能：获取电平的状态
 * @参数：gpio：gpio编号
 * @返回值：1高电平，0低电平
 * @备注：无
*/
int gpio_get_value(unsigned gpio)

/**
 * @功能：释放gpio
 * @参数：gpio：gpio编号
 * @返回值：无
 * @备注：无
*/
void gpio_free(unsigned gpio)

# 7. 中断子系统

## 7.1 中断
中断是基于硬件实现的，有无linux内核中断的处理的流程都是一样的。

### 7.1.1 中断的产生
异常处理流程（4 大步，3 小步）
1. 将cpsr保存到spsr中
2. 设置cpsr
    1. 设置为arm核为svc模型
    2. 如果有必要进制相应的中断
    3. 设置为对应的异常模式
3. 保存LR
4. 设置PC到对应的执行的位置

### 7.1.2 接口

In [ ]:
/**
 * @功能：获取软中断号
 * @参数：np：设备树节点指针，index：中断号索引
 * @返回值：成功返回软中断号，失败返回0
 * @备注：无
*/
unsigned int irq_of_parse_and_map(struct device_node *np,int index)

#include <linux/interrupt.h>
/**
 * @功能：注册中断
 * @参数：irq：软中断号（设备树），handler：中断处理函数，flags：中断触发方式，
 *        name：中断的名字，dev：向中断处理函数传递的参数
 *      @handler:中断处理函数
 *         irqreturn_t key_handle(int irqno, void *dev)
 *         {
 *              //中断处理
 *              return  IRQ_NONE; //中断没有正常处理
 *              return  IRQ_HANDLED //中断正常处理函数
 *         }
 *      @flags:中断触发方式
 *          #define IRQF_TRIGGER_RISING	0x00000001
 *          #define IRQF_TRIGGER_FALLING	0x00000002
 *          #define IRQF_TRIGGER_HIGH	0x00000004
 *          #define IRQF_TRIGGER_LOW	0x00000008
 *          #define IRQF_SHARED		0x00000080
 *      @name:中断的名字  cat    /proc/interrupts
 * @返回值：成功返回0，失败返回错误码
 * @备注：中断处理函数的格式：irqreturn_t key_handle(int irqno, void *dev)
*/
int request_irq(unsigned int irq, irq_handler_t handler, unsigned long flags,
     const char *name, void *dev)
/**
 * @功能：释放中断
 * @参数：irq：软中断号，dev_id：向中断处理函数传递的参数
 * @返回值：无
*/
const void *free_irq(unsigned int irq, void *dev_id)

## 7.2 定时器
jiffies: 内核时钟节拍数， 在 linux 内核中它一直增加。
内核中的定时器频率可通过 make menuconfig -> kernel hacking -> timer frequency 来设置。宏 HZ
周期：10ms

### 7.2.1 定时器的使用

In [ ]:
// 1.分配定时器的对象
struct timer_list {
    struct hlist_node entry;  //构成内核链表
    unsigned long  expires; //定时到期的时间  jiffies+100
    void   (*function)(struct timer_list *); //定时器初始函数
    u32			flags; //定时器的标志位 0
};
struct timer_list mytimer;

// 2.初始化定时
mytimer.expires = jiffies + 100;
/**
 * @功能：设置定时器的处理函数
 * @参数：定时器对象，定时器处理函数，定时器标志位
 * @返回值：无
*/
timer_setup(&mytimer, mytimer_function, 0);

// 3.启动定时器
/**
 * @功能：启动定时器
 * @参数：timer_list对象
 * @返回值：无
*/
void add_timer(struct timer_list *timer);

// 3.1 再次启动定时器
/**
 * @功能：再次启动定时器
 * @参数：timer : timer_list对象，expires : 定时器到期时间
 * @返回值：无
*/
int mod_timer(struct timer_list *timer, unsigned long expires);

// 4.删除定时器
/**
 * @功能：删除定时器
 * @参数：timer : timer_list对象
 * @返回值：无
*/
int del_timer(struct timer_list * timer);

# 8. platform 总线驱动

# 9. IIC 总线驱动

## 9.1 IIC 信号
起始信号：当scl为高电平的时候，sda从高到低的跳变
停止信号：当scl为高电平的时候，sda从低到高的跳变
应答信号：在第9个时钟周期的时候，sda这根线是低电平
非应答信号：在第9个时钟周期的时候，sda这根线是高电平
## 9.2 IIC 数据收发协议
写：`start+(7bit从机地址+1bit写0)+ack+(8bit/16bit寄存器地址)+ack+(8bit/16bit数据)+ack+stop`   
读：`start+(7bit从机地址+1bit写0)+ack+(8bit/16bit寄存器地址)+ack+start+(7bit从机地址+1bit读1)+ack+(从机给主机返回的数据8bit/16bit)+No Ack+stop`
## 9.3 速率
100KHz(低速) 400KHz(全速) 3.4MHz(高速)   
stm32mp157a的i2c控制器的最高速率1MHz   
stm32mp157a是异构核，cortex-a7的速率是650MHz，cortex-m4的速率是209MHz
## 9.4 特点
i2c是一个同步，串行的，具备应答机制的半双工的总线协议
## 9.5 编写流程

In [ ]:
// 1.分配并初始化对象
struct i2c_driver {
    int (*probe)(struct i2c_client *client, const struct i2c_device_id *id);
    //匹配成功执行的函数
    int (*remove)(struct i2c_client *client);
    //分离的时候执行的函数
    struct device_driver driver;
    //父类
    const struct i2c_device_id *id_table;
    //1. idtable的匹配方法
}
struct device_driver {
    const char  *name; //i2c没有名字匹配方式，一定要填充
    const struct of_device_id *of_match_table;//2.设备树匹配
};

// 2. 注册
#define i2c_add_driver(driver) i2c_register_driver(THIS_MODULE, driver)   

// 3. 注销
/**
 * @功能：注销i2c_driver
 * @参数：driver：i2c_driver对象
 * @返回值：无
*/
void i2c_del_driver(struct i2c_driver *driver);

// 4. 一键注册注销的宏
/**
 * @功能：一键注册i2c_driver
 * @参数：i2c_driver的变量名
 * @返回值：无
*/
module_i2c_driver(i2c_driver的变量名);

// 消息发送
/**
 * @功能：i2c发送消息
 * @参数：adap：i2c_adapter对象，msgs：i2c_msg对象，num：消息个数
 * @返回值：成功返回发送的消息个数，失败返回负数
 * @备注：i2c_msg对象包含了发送的地址，数据，长度等信息
*/
int i2c_transfer(struct i2c_adapter *adap, struct i2c_msg *msgs, int num);

### 9.5.1 设备树

In [ ]:
// 控制器设备树节点
i2c1: i2c@40012000 {
    compatible = "st,stm32mp15-i2c";
    reg = <0x40012000 0x400>;
    interrupt-names = "event", "error";
    interrupts-extended = <&exti 21 IRQ_TYPE_LEVEL_HIGH>,
    <&intc GIC_SPI 32 IRQ_TYPE_LEVEL_HIGH>;
    clocks = <&rcc I2C1_K>;
    resets = <&rcc I2C1_R>;
    #address-cells = <1>;   //修饰子节点 reg地址个数
    #size-cells = <0>;      //修饰子节点 reg地址个数
    dmas = <&dmamux1 33 0x400 0x80000001>,
    <&dmamux1 34 0x400 0x80000001>;
    dma-names = "rx", "tx";
    power-domains = <&pd_core>;
    st,syscfg-fmp = <&syscfg 0x4 0x1>;
    wakeup-source;
    i2c-analog-filter;
    status = "disabled";  //控制器状态
};

// 自己的设备树节点
// linux-5.10.61/Documentation/devicetree/bindings/i2c
&i2c1{
    pinctrl-names   = "default","sleep";
    pinctrl-0   = <&i2c1_pins_b>;      //工作状态：gpiof14和gpiof15复用为AF5功能
    pinctrl-1   = <&i2c1_sleep_pins_b>;//休眠状态：将管脚设置为ANALOG 
    clock-frequency = <100000>;        //i2c速率，默认就是100Khz
    i2c-scl-rising-time-ns = <185>;    //上升沿时间
    i2c-scl-falling-time-ns = <20>;    //下降沿时间
    status = "okay";                  //控制器使能
    si7006@40{                        //si7006节点
        compatible = "hqyj,si7006";   //和自己驱动匹配的名字
        reg = <0x40>;                 //从机地址
    };
};

### 9.5.2 驱动消息封装

In [ ]:
// 驱动消息相关结构体

// 1. i2c_client结构体
// 当I2C的设备驱动和I2C的总线驱动匹配成功后，就会创建I2C_client结构体，
// 这个结构体使用来携带信息的。比如从机地址，总线驱动的对象等都会传递给
// IIC的设备驱动。
struct i2c_client {
    unsigned short flags;  //i2c相关的标志位
    I2C_CLIENT_PEC  /* Use Packet Error Checking */
    I2C_CLIENT_TEN  /* we have a ten bit chip address */
    I2C_CLIENT_SLAVE /* we are the slave */
    I2C_CLIENT_SCCB  /* Use Omnivision SCCB protocol */

    unsigned short addr; //从机地址，从设备树中解析得到的reg = <0x40>
    char name[I2C_NAME_SIZE]; //名字
    struct i2c_adapter *adapter;//i2c总线驱动对象（控制器驱动对象）
    struct device dev;  // dev.of_node 设备树节点
    int irq;    //软中断号
};

// 2. i2c_msg结构体
// i2c_msg结构体是消息结构体，设备驱动和总线驱动进行数据传输的时候，
// 就是使用的这个i2c_msg结构体。
struct i2c_msg {
    __u16 addr;   //从机地址  client->addr
    __u16 flags;  //读写标志位 0写    1读
    __u16 len;   //消息的长度
    __u8 *buf;   //指向消息的首地址
};

### 9.5.3 读写消息函数封装

In [ ]:
// 写
int i2c_write_reg(unsigned char reg,unsigned char val)
{
    int ret;
    char w_buf[2] = {reg,val};
    //1.封装消息
    struct i2c_msg w_msg = {
        .addr = client->addr,
        .flags = 0,
        .len = 2,
        .buf = w_buf,
    };
    //2.发送消息
    ret = i2c_transfer(client->adapter,&w_msg,1);
    if(ret != 1){
        printk("i2c_transfer write error\n");
        return -EAGAIN;
    }
    return 0;
}

// 读
int i2c_read_reg(unsigned char reg)
{
    int ret;
    char r_buf[] = {reg};
    unsigned char data;
    //1.封装消息
    struct i2c_msg r_msg[2] = {
        [0] = {
            .addr = client->addr,
            .flags = 0,
            .len = 1,
            .buf = r_buf,
        },
        [1] = {
            .addr = client->addr,
            .flags = 1,
            .len = 1,
            .buf = &data,
        },
    };
 //2.发送消息
    ret = i2c_transfer(client->adapter,r_msg,ARRAY_SIZE(r_msg));
    if(ret != ARRAY_SIZE(r_msg)){
        printk("i2c_transfer read error\n");
        return -EAGAIN;
    }
    return data;
}